# Quick Test of New Simulation Features

A faster test using shorter simulation duration (7 days instead of 30).

In [ ]:
import numpy as np
import time
import os

from pft_fem import (
    MNIAtlasLoader,
    MRISimulator,
    TumorParameters,
    SolverConfig,
    MRISequence,
    save_nifti,
    get_regional_properties,
    BrainTissue,
)

## 1. Test Regional Material Properties

In [ ]:
print("Regional Material Properties (Literature-based)")
print("=" * 50)

regions = [
    ("Cerebellum", np.array([0.0, -60.0, -30.0])),
    ("Brainstem", np.array([0.0, -35.0, -45.0])),
]

for name, coords in regions:
    props_gm = get_regional_properties(coords, BrainTissue.GRAY_MATTER)
    props_wm = get_regional_properties(coords, BrainTissue.WHITE_MATTER)
    
    print(f"\n{name}:")
    print(f"  GM:  E={props_gm.young_modulus:.0f} Pa, nu={props_gm.poisson_ratio:.2f}")
    print(f"  WM:  E={props_wm.young_modulus:.0f} Pa, nu={props_wm.poisson_ratio:.2f}")

## 2. Load Atlas and Configure

In [ ]:
print("Loading MNI Atlas...")
start = time.time()
atlas = MNIAtlasLoader().load()
print(f"Loaded in {time.time() - start:.1f}s")

# Configure with ALL new features
tumor_params = TumorParameters(
    initial_radius=3.0,
    proliferation_rate=0.05,
    diffusion_rate=0.01,
    fa_anisotropy_factor=6.0,           # NEW
    use_volume_preserving_mass_effect=True,  # NEW
    use_adaptive_stepping=True,         # NEW
    dt_min=1.0,
    dt_max=3.0,
    use_ventricular_compliance=True,    # NEW
)

print(f"\nNew features enabled:")
print(f"  - FA anisotropy factor: {tumor_params.fa_anisotropy_factor}")
print(f"  - Volume-preserving mass effect: {tumor_params.use_volume_preserving_mass_effect}")
print(f"  - Adaptive stepping: {tumor_params.use_adaptive_stepping}")
print(f"  - Ventricular compliance: {tumor_params.use_ventricular_compliance}")

## 3. Create Simulator

In [ ]:
print("Creating simulator...")
start = time.time()

simulator = MRISimulator(
    atlas,
    tumor_params=tumor_params,
    solver_config=SolverConfig.default(),
    use_default_solver=True,
)

print(f"Created in {time.time() - start:.1f}s")

## 4. Run 7-Day Simulation

In [ ]:
print("Running 7-day simulation...")
start = time.time()

result = simulator.run_full_pipeline(
    duration_days=7,
    sequences=[MRISequence.T1],
    verbose=True,
)

elapsed = time.time() - start
print(f"\nCompleted in {elapsed:.1f}s ({elapsed/60:.1f} min)")

## 5. Results

In [ ]:
states = result.tumor_states
print(f"Time steps: {len(states)-1}")
print(f"Final time: {states[-1].time:.1f} days")

# Adaptive stepping analysis
if len(states) > 2:
    dts = np.diff([s.time for s in states])
    print(f"dt range: [{min(dts):.2f}, {max(dts):.2f}] days")

print(f"\nTumor volume: {states[0].current_volume:.0f} -> {states[-1].current_volume:.0f} mm³")
print(f"Max displacement: {np.max(np.linalg.norm(states[-1].displacement, axis=1)):.2f} mm")
print(f"Tumor mask voxels: {np.sum(result.tumor_mask)}")

## 6. Save Output

In [ ]:
os.makedirs("simulation_output", exist_ok=True)
save_nifti(result.mri_images['T1'], atlas.affine, "simulation_output/quick_test_T1.nii.gz")
save_nifti(result.tumor_mask.astype(np.float32), atlas.affine, "simulation_output/quick_test_mask.nii.gz")
print("Saved to simulation_output/")

In [ ]:
print("\n" + "="*50)
print("SUCCESS: All new features working!")
print("="*50)